# Email Marketing Campaign Data

## About

### Objective
Generate sample data for an email marketing campaign dashboard using Faker library

### Dataset requirements
- approx. 2 years of data (Jan 2022 - May 2024)
- approx. 40,000 customers 
- 6 different email types: welcome, promotion, newsletter, new product, seasonal offer, re-engagement
- emails sent in daily intervals
- varying probabilities for email interactions; opened, clicked, conversion

### Reference
Code used for reference: Email Marketing Campaign Dashboard by Marius Nikiforovas
https://github.com/Marius321/Email-Marketing-Campaign-Dashboard/blob/main/generate_emails.py
Reorganized and modified to fit my requirements for the dataset.

### Method
1. Import the necessary libraries
2. Write function to generate emails sent dataset
    - Define email names
    - Define probabilities for interactions
    - Define customer names and account numbers
    - Define time range
    - Define bounce probability
    - Generate emails
    - Append data to list
3. Run function and create data frame
4. Check and evaluate dataset
5. Assign values of 0 or 1 = email date is null or not null, to make aggregation easier
6. Write final dataset to csv to be used in Tableau

## Import libraries

In [1]:
import pandas as pd
import numpy as np
import random
from faker import Faker
from datetime import datetime, timedelta # to represent difference between dates
pd.set_option("max_columns", 50) # maximum amount of columns for data frame

## Generate dataset

### Seed random number generator

In [2]:
# Seed the random number generator
random.seed(4233)
np.random.seed(4233)
Faker.seed(4233)
faker = Faker('en_US') # set locale to US

### Write function to generate sent emails data

In [3]:
def generate_emails_data(num_customers=43200, num_emails=6):
    # Email names corresponding to email numbers
    email_name = {
        1: "Email 1: Welcome",
        2: "Email 2: Promotion",
        3: "Email 3: Newsletter",
        4: "Email 4: New Product",
        5: "Email 5: Seasonal Offer",
        6: "Email 6: Re-Engagement"
    }
    # Probabilities for opening email
    open_probabilities = {
        1: 0.675,
        2: 0.639,
        3: 0.458,
        4: 0.513,
        5: 0.478,
        6: 0.389
    }
    # Probabilities for clicking on email
    click_probabilities = {
        1: 0.472,
        2: 0.385,
        3: 0.11,
        4: 0.24,
        5: 0.257,
        6: 0.167
    }
    # Probabilities for conversion after clicking
    conversion_probabilities = {
        1: 0.079,
        2: 0.149,
        3: 0.129,
        4: 0.037,
        5: 0.082,
        6: 0.065
    }
    
    emails_sent = [] # List to store generated emails sent data

    for i in range(num_customers):
        customer_name = faker.name()
        customer_number = faker.random_number(digits=8)
        
        # Define time range for 2 years of email interactions
        start_time = datetime(year=2022, month=1,day=1)
        end_time = datetime(year=2024, month=5,day=31)
        
        # Generate a random email interaction time
        email_datetime = faker.date_time_between_dates(datetime_start=start_time, datetime_end=end_time)
        
        # Send new email in daily intervals
        for j in range(1, num_emails + 1):
            sent_datetime = email_datetime + timedelta(days=j-1)
            
            # Set all dates to none at first
            if start_time <= sent_datetime <= end_time:
                bounce_datetime = None
                open_datetime = None
                click_datetime = None
                transaction_datetime = None
                transaction_amount = None
                
                #Define bounce probability
                bounce_probability = random.random()
                if bounce_probability <= 0.016:
                    bounce_datetime = sent_datetime + timedelta(minutes=random.randint(0, 1))
                #if not bounced continue with interactions and probabilites
                else:
                    open_probability = random.random()
                    if open_probability <= open_probabilities[j]:
                        open_datetime = sent_datetime + timedelta(minutes=random.randint(1, 59))
                        click_probability = random.random()
                        if click_probability <= click_probabilities[j]:
                            click_datetime = open_datetime + timedelta(minutes=random.randint(1, 59))
                            conversion_probability = random.random()
                            if conversion_probability <= conversion_probabilities[j]:
                                transaction_amount = round(random.uniform(600, 1500), 2)
                                transaction_datetime = click_datetime + timedelta(days=random.randint(1, 30))  # Random transaction date within 30 days after the click
                
                # Append generated data to the list
                emails_sent.append({
                    "customer_name": customer_name,
                    "customer_number": customer_number,
                    "email_name": email_name[j],
                    "sent_date": sent_datetime.strftime('%Y-%m-%d %H:%M:%S'),
                    "open_date": open_datetime.strftime('%Y-%m-%d %H:%M:%S') if open_datetime else None,
                    "click_date": click_datetime.strftime('%Y-%m-%d %H:%M:%S') if click_datetime else None,
                    "bounce_date": bounce_datetime.strftime('%Y-%m-%d %H:%M:%S') if bounce_datetime else None,
                    "transaction_date": transaction_datetime.strftime('%Y-%m-%d %H:%M:%S') if transaction_datetime else None,
                    "transaction_amount": transaction_amount
                })
    
    return emails_sent

### Run function to generate data

In [4]:
emails_data = generate_emails_data()

### Create dataframe

In [5]:
emails_df = pd.DataFrame(emails_data)

### Check data frame

In [6]:
emails_df.shape

(258399, 9)

In [7]:
emails_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258399 entries, 0 to 258398
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   customer_name       258399 non-null  object 
 1   customer_number     258399 non-null  int64  
 2   email_name          258399 non-null  object 
 3   sent_date           258399 non-null  object 
 4   open_date           133767 non-null  object 
 5   click_date          39102 non-null   object 
 6   bounce_date         4112 non-null    object 
 7   transaction_date    3575 non-null    object 
 8   transaction_amount  3575 non-null    float64
dtypes: float64(1), int64(1), object(7)
memory usage: 17.7+ MB


In [8]:
emails_df.head()

,customer_name,customer_number,email_name,sent_date,open_date,click_date,bounce_date,transaction_date,transaction_amount
0,William Vang,73760787,Email 1: Welcome,2022-06-05 01:08:10,None,None,None,None,NaN
1,William Vang,73760787,Email 2: Promotion,2022-06-06 01:08:10,2022-06-06 01:18:10,2022-06-06 01:38:10,None,None,NaN
2,William Vang,73760787,Email 3: Newsletter,2022-06-07 01:08:10,None,None,None,None,NaN
3,William Vang,73760787,Email 4: New Product,2022-06-08 01:08:10,2022-06-08 02:01:10,2022-06-08 02:59:10,None,None,NaN
4,William Vang,73760787,Email 5: Seasonal Offer,2022-06-09 01:08:10,None,None,None,None,NaN


In [9]:
emails_df.sort_values("sent_date").head()

,customer_name,customer_number,email_name,sent_date,open_date,click_date,bounce_date,transaction_date,transaction_amount
64766,Amanda Rogers,87422282,Email 1: Welcome,2022-01-01 00:09:52,None,None,None,None,NaN
249007,Felicia Jones MD,93788690,Email 1: Welcome,2022-01-01 00:40:02,2022-01-01 01:38:02,None,None,None,NaN
212227,Shaun Kline,48002446,Email 1: Welcome,2022-01-01 00:51:10,None,None,None,None,NaN
145936,Elizabeth Murphy,81854109,Email 1: Welcome,2022-01-01 01:00:16,2022-01-01 01:30:16,2022-01-01 02:06:16,None,None,NaN
62007,Marissa Harris,76124123,Email 1: Welcome,2022-01-01 01:52:07,2022-01-01 02:06:07,None,None,None,NaN


In [10]:
emails_df[["email_name", "sent_date","open_date", "click_date", "bounce_date","transaction_date"]].groupby("email_name").count().sort_values("email_name")

,sent_date,open_date,click_date,bounce_date,transaction_date
email_name,,,,,
Email 1: Welcome,43200,28869,13459,666,1028
Email 2: Promotion,43155,27158,10429,688,1503
Email 3: Newsletter,43090,19339,2193,727,265
Email 4: New Product,43037,21793,5250,671,191
Email 5: Seasonal Offer,42983,20209,5143,725,428
Email 6: Re-Engagement,42934,16399,2628,635,160


### Assign 0 or 1 based on dates for counting

In [11]:
# Create new columns and add 1 if date not null, else 0
emails_df["email_sent"] = np.where(emails_df.sent_date.notnull(), 1, 0)
emails_df["email_opened"] = np.where(emails_df.open_date.notnull(), 1, 0)
emails_df["email_clicked"] = np.where(emails_df.click_date.notnull(), 1, 0)
emails_df["email_bounced"] = np.where(emails_df.bounce_date.notnull(), 1, 0)
emails_df["transaction_no"] = np.where(emails_df.transaction_date.notnull(), 1, 0)

In [12]:
# Check result
emails_df.sample(5)

,customer_name,customer_number,email_name,sent_date,open_date,click_date,bounce_date,transaction_date,transaction_amount,email_sent,email_opened,email_clicked,email_bounced,transaction_no
202042,Allison Jackson,45715707,Email 1: Welcome,2023-11-12 09:49:19,None,None,None,None,NaN,1,0,0,0,0
34174,Michele Stein,22307554,Email 1: Welcome,2022-10-14 18:25:33,2022-10-14 19:03:33,2022-10-14 19:31:33,None,None,NaN,1,1,1,0,0
203473,David Armstrong,9410002,Email 3: Newsletter,2022-05-21 06:48:59,None,None,None,None,NaN,1,0,0,0,0
61364,Tracey Price,40322978,Email 6: Re-Engagement,2024-01-12 06:29:29,None,None,None,None,NaN,1,0,0,0,0
5166,Scott Smith,65540704,Email 2: Promotion,2024-02-27 06:17:36,None,None,None,None,NaN,1,0,0,0,0


In [13]:
# Check if numbers match
emails_df[["email_name","email_sent", "email_opened", "email_clicked", "email_bounced", "transaction_no"]].groupby("email_name").sum().sort_values("email_name")

,email_sent,email_opened,email_clicked,email_bounced,transaction_no
email_name,,,,,
Email 1: Welcome,43200,28869,13459,666,1028
Email 2: Promotion,43155,27158,10429,688,1503
Email 3: Newsletter,43090,19339,2193,727,265
Email 4: New Product,43037,21793,5250,671,191
Email 5: Seasonal Offer,42983,20209,5143,725,428
Email 6: Re-Engagement,42934,16399,2628,635,160


## Write final dataset to csv

In [15]:
# emails_df.to_csv("Email_Campaign_Data.csv", index = False)

## Further resources and notes
- Articles about email campaigns:
    - https://mailchimp.com/resources/email-marketing-campaign-examples/
    - https://www.sender.net/blog/email-campaign/
- Faker documentation https://faker.readthedocs.io/en/master/index.html
- Numpy where
    - https://numpy.org/doc/stable/reference/generated/numpy.where.html
    - https://www.programiz.com/python-programming/numpy/methods/where
    